<div>
    <img src="http://www.ient.rwth-aachen.de/cms/uploads/images/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

In [ ]:
# Copyright 2021 Institut für Nachrichtentechnik, RWTH Aachen University

#Numpy,Sys, Matplotlib Imports, display widgets correctly 
import sys
sys.path.insert(0,'./Bilder')
sys.path.insert(0,'../ient_python')
%matplotlib widget
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.pyplot import figure, draw, pause, close
from matplotlib.widgets import RectangleSelector

#iPython Imports
from ipywidgets import widgets,interact
import IPython.display as ip
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
from IPython.display import Video

#RWTH imports
import rwth_nb.plots.mpl_decorations as rwth_plt
import rwth_nb.misc.feedback as rwth_feedback

#Scikit-Image, cv2 Imports
from skimage.filters import threshold_otsu
from skimage.morphology import label, square,binary_erosion, binary_dilation, disk
from skimage.measure import regionprops
from skimage.draw import rectangle_perimeter
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.transform import rotate, rescale
from scipy.ndimage import binary_fill_holes
from cv2 import warpAffine, getRotationMatrix2D, resize, INTER_LINEAR

# Suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

plt.close('all')
plt.rcParams.update({'figure.max_open_warning': 0})

## Teil 1: Klassifikation mit Formmerkmalen
In diesem Teil des Projekts können einige Schritte zur Klassifikation von Objekten in Bildern mittels Formmerkmalen ausprobiert werden.
Für eine solche Klassifikation muss ein geladenes Bild zunächst binarisieren, mögliches Rauschen im Bild reduzieren und der Hintergrund segmentiert (erkannt) werden. Anschließend müssen die einzelnen Objekte des Bildes voneinander separiert und die Merkmale berechnet werden.

Alle benötigten Funktionen, können hier gefunden werden:

__[Befehlsreferenz](Befehlsreferenz_Schueleruni.ipynb)__


<div class="alert rwth-topic">
    
## Schritt 1: Binarisierung

Als Binarisierung bezeichnet man die Umwandlung des Bildes von einem beliebigen Ursprungsformat in ein reines Schwarz/Weiß Bild ohne Graustufen.
Ziel der Binarisierung ist die Segmentierung eines Bildes in Vorder- und Hintergrundregionen. Dabei soll der Vordergrundregionen der Wert 1 und der Hintergrundregion der Wert 0 zugeordnet werden. Nach der Binarisierung hat also jedes Pixel des Bildes also nur noch den Wert 1 (Vordergrund) oder 0 (Hintergrund).

Durch Ausführen der nächsten Zelle, können Sie sich zunächst das Ausgangsbild ansehen.
    
</div>

In [ ]:
%matplotlib inline

#Laden des Bildes
img = imread("Bilder/Buchstaben.png");
#Anzeigen des geladenen Bildes:
fig, ax = plt.subplots(1,1, figsize=(12,12));
ax.imshow(img, cmap='gray');
ax.set_axis_off();
ax.set_title('Verrauschtes Bild mit Buchstaben', fontsize=18);
plt.show()

%matplotlib widget
%matplotlib widget

<div class="alert rwth-subtopic">
    
### Schritt 1.1: Manuelle Segmentierung
Eine der einfachsten Möglichkeiten Vorder- und Hintergrund zu trennen ist ein Schwellwert. Das bedeutet, alle Pixel die heller sind als dieser Wert, werden dem einen Teil, alle anderen dem anderen Teil zugeordnet. Die Schwierigkeit dabei ist, diesen Schwellwert sinnvoll festzulegen. 

Wenn der nachfolgende Block ausgeführt wurde, können Sie sich das Resultat für verschiedene Werte einmal ansehen indem Sie den Regler darunter verschieben. Zudem wird auch das Histogram (die Häufigkeitsverteilung) der Grauwerte im Ursprungsbild gezeigt, da dies Hinweise für die Wahl eines sinnvollen Wertes liefert.

</div>

In [ ]:
from ipywidgets import interact, fixed, IntSlider, HBox, Layout, Output, VBox

fig1, ax = plt.subplots(1,2,figsize=(15, 5))

#Hier wird das Graustufenbild aus dem RGB Farbraum erzeugt
img_gray = rgb2gray(img)

#Hier kommt Ihre Lösung zu Aufgabe 1.2:
img_gray_hist = img_gray.ravel()
histogram = ax[0].hist(img_gray_hist, bins=32)
ax[0].set_title("Histogramm")
ax[0].set_xlabel("Pixelwert")
ax[0].set_ylabel("Häufigkeit")
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)


@widgets.interact(t=widgets.IntSlider(min=50, max=240, continuous_update=True, description='Schwellwert t'))
def update_plot(t):
    if ax[1].lines: # check if lines exist
        ax[1].set_data(img_gray>t);
        ax[1].draw()
    else:
        ax[1].imshow(img_gray>t, cmap= 'gray')
        ax[1].set_title("Finden der Binarisierungsschwelle")
        ax[1].axis('off')


<div class="alert rwth-subtopic">
    
### Schritt 1.2: Binarisierung mit Otsu
Wenn Buchstaben und Zahlen automatisch erkannt werden sollen, kann nicht jedes Mal ein Mensch erst den richtigen Schwellwert händisch festlegen. Dazu gibt es Verfahren, die diesen Wert automatisch an das Bild anpassen. Eines der grundlegensten ist die Methode von Otsu, bei der die Varianz innerhalb der beiden Bereiche minimiert wird. 

In der nächsten Zelle sollen Sie diese Methode einmal selber programieren.
Wenn Sie das Resultat in der Variable 'thresh_otsu' speichern, können Sie sich das resultierende Bild durch Ausführen der darauffolgenden Zelle ansehen.

</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 1.2:
thresh_otsu = 0

In [ ]:
%matplotlib inline

#Binarisieren des Bildes mit dem ermittelten Schwellwert nach Otsu: Alle Werte, die größer als der Schwellwert sind werden zu 1 gesetzt, die anderen zu 0
img_bin = img_gray>thresh_otsu

#Laden des Bildes
img = imread("Bilder/Buchstaben.png");
#Anzeigen des geladenen Bildes:
fig, ax = plt.subplots(1,1, figsize=(12,12));
ax.imshow(img_bin, cmap='gray');
ax.set_axis_off();
ax.set_title('Binarisiertes Bild nach Otsu', fontsize=18);
plt.show()

%matplotlib widget
%matplotlib widget

<div class="alert rwth-topic">
    
## Schritt 2: Rauschreduktion 
Ziel dieses Schrittes ist es, dass im Binärbild vorhandene Rauschen (Pixel, die dem Vordergrund zugerechnet werden, aber eigentlich zum Hintergrund gehören) so weit wie möglich zu reduzieren. Dies kan mit Hilfe der morphologischer Operationen erreicht werden zu denen unter anderem die  __Erosion__ und __Dilatation__ gehören.  

</div>

<div class="alert rwth-subtopic">
      
         
### Schritt 2.1: Binäre Erosion
Mit hilfe der Erosion können störende Bildpunkte entfernt werden. Das Ergebniss hängt dabei sehr vom verwendeten Strukturelemente (der verwendeten geometrischen Form, engl. Kernel) ab. Mögliche Strukturelemente sind in der 
__[Befehlsreferenz](Befehlsreferenz_Schueleruni.ipynb)__ aufgelistet.
    
Indem Sie die Strukturelemente 'kernel1' und 'kernel2' in der nachfolgenden Zelle definieren, können Sie verschiedene Formen und Größen ausprobieren und vergleichen. Das jeweilige Ergebnis können Sie durch Ausführen der darauffolgenden Zelle ansehen.
           
</div>

In [ ]:
#Hier kommt Ihre Lösung zu Aufgabe 2.1:
kernel1 = #...
kernel2 = #...


In [ ]:
%matplotlib inline
#Invertierung des Bildes
img_tmp = ~img_bin

#Anwendung der binären Erosion mit erstellten Strukturelementen
img_erode1 = binary_erosion(img_tmp,kernel1) 
img_erode2 = binary_erosion(img_tmp,kernel2)

#Plotten der binären Erosionen aus 2.1:
fig, ax = plt.subplots(1,2,sharex='all', sharey='all', figsize=(20,20))
ax[0].imshow(img_erode1,cmap='gray')
ax[0].set_title("Erodiertes Bild mit Kernel 1", fontsize=20)
ax[1].imshow(img_erode2,cmap='gray')
ax[1].set_title("Erodiertes Bild mit Kernel 2", fontsize=20)

for axs in ax.flat:
    axs.set_axis_off()
    
    
plt.show()

%matplotlib widget
%matplotlib widget

<div class="alert rwth-subtopic"> 
    
### Schritt 2.2: Rekonstruktion
Die Erosion im vorhergehenden Schritt hat neben dem Rauschen auch zum Objekt gehörende Bildpunkte entfernt. Diese können teilweise durch eine Dilatation mit dem selbe Strukturelement wiederhergestellt werden. Das Resultat können Sie sich wieder durch Ausführen der nächsten Zelle ansehen. Bleiben Sie bei Ihrer Beurteilung, welche Form am besten geeignet ist?
    
__Hinweis__: In den nachfolgen Aufgaben wird mit dem ersten (linken) Bild weitergearbeitet. 
        
</div>

In [ ]:
%matplotlib inline
#Anwendung der binären Dilatation mit erstellten Strukturelementen
img_dil1 = binary_dilation(img_erode1, kernel1) 
img_dil2 = binary_dilation(img_erode2, kernel2) 
#Plotten der binären Dilatationen aus 2.2:

fig, ax = plt.subplots(1,2,sharex='all', sharey='all', figsize=(20,20))
ax[0].imshow(img_dil1,cmap='gray')
ax[0].set_title("Dilatiertes Bild mit Kernel 1", fontsize=20)
ax[1].imshow(img_dil2,cmap='gray')
ax[1].set_title("Dilatiertes Bild mit Kernel 2", fontsize=20)

for axs in ax.flat:
    axs.set_axis_off()    
    
plt.show()

%matplotlib widget
%matplotlib widget

<div class="alert rwth-topic">
   
## Schritt 3: Labeling und Ausschneiden 

Um die einzelnen Objekte im Bild beschreiben zu können, müssen die Objekte zunächst grob voneinander getrennt werden.
Im Fall von Druckschrift reicht meist schon ein Rechteck um jeweils zusammenhängende Teile des Vordergrunds auszuschneiden. 
    
Das Ergebnis so wie zwei Zwischenschritte werden nach Ausführung der nächsten Zelle angezeigt.
      
</div>

<div class="alert rwth-topic">
    
## Schritt 4: Merkmale und Invarianzen
Eine beliebtes Merkmal um die From eines Objekts, also hier eines Buchstabens, zu beschreiben ist die Homogenität. Diese ist als Verhältnis des Umfangs (U) zum Quadrat zur Fläche (A) definiert: H=U^2/A 
Man kann sich dieses Merkmal auch als Kehrwert der Rundheit eines Objectes vorstellen. 

Nach Ausführen der nächsten Zelle wird ein Beispielbild und dessen Homogenität angezeigt. Mit Hilfe der Regler können die Bilder gedreht oder skaliert werden. Welchen Effekt hat das auf die Homogenität? Kann man die Buchstaben anhand ihrer Homogenität eindeutig identifizieren?
</div>

In [ ]:
kernelX = square(3)

def ient_homogenity(contour,image):
    homogenity = (np.sum(contour)*np.sum(contour))/np.sum(image)
    print('Homogenität: %.2f' %homogenity)
    

#load the cropped image embedded in bigger black img to avoid bound-crops and overlapping by neighboured characters
a_in_black = (imread("Bilder/A_in_black.png")).astype(bool)

#Rotate image correctly
def rotate_image(mat, angle):
    height, width = mat.shape[:2] 
    image_center = (width/2, height/2)

    rotation_mat = getRotationMatrix2D(image_center, angle, 1.)

    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    rotated_mat = warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat


fig, ax = plt.subplots(1,2,figsize=(10, 6))

#Rotate Slider
@widgets.interact(d=widgets.IntSlider(min=0, max=360, continuous_update=False))
def update_rotation(d):
        rotated_a = label(rotate_image(a_in_black.astype(np.double), d))
        regions = regionprops(rotated_a)
        boxes = np.array([label['BoundingBox']
                  for label in regions])
        rotated_a_cropped = rotated_a[boxes[0][0]-1:boxes[0][2]+1, boxes[0][1]-1:boxes[0][3]+1]
    
        if ax[0].lines: # check if lines exist
            ax[0].set_data(rotated_a_cropped);
            ax[0].draw()

        else:
            ax[0].imshow(rotated_a_cropped, cmap= 'gray')
            ax[0].set_title("Rotation des Buchstaben")
            ax[0].axis('off')
            
        print('Winkel: ',d, '°')
        rotated_filled = binary_fill_holes(rotated_a_cropped)
        rotated_contour = rotated_filled.astype(np.double) - binary_erosion(rotated_filled, kernelX)
        ient_homogenity(rotated_contour,rotated_filled)

#Resize Slider
@widgets.interact(r=widgets.FloatSlider(min=0.01, max=5, step=0.05,continuous_update=False))
def update_resize(r):
        # parameter for resizing
        width = int(a_in_black.shape[1] * r)
        height = int(a_in_black.shape[0] * r)
        dsize = (width, height)
        # resizing
        resized_image= resize(a_in_black.astype(float),dsize,interpolation = INTER_LINEAR)#INTER_NEAREST
        # crop to get A with little margin and cast to boolean
        resized_image_a = label(resized_image.astype(float))
        regions = regionprops(resized_image_a)
        boxes = np.array([label['BoundingBox']
                for label in regions])
        resized_image_cropped = resized_image_a[boxes[0][0]-1:boxes[0][2]+1, boxes[0][1]-1:boxes[0][3]+1].astype(bool)
        # plotting
        if ax[1].lines: # check if lines exist, if so: just update data
            ax[1].set_data(resized_image_cropped);
            ax[1].draw()
        else: # plot whole figure
            ax[1].imshow((resized_image_cropped), cmap= 'gray')
            ax[1].set_title("Skalierung des Buchstaben")
            ax[1].axis('off')
        print('Skalierungsfaktor: ',r)
        # homogenity
        scaled_filled = binary_fill_holes(resized_image)
        scaled_contour = scaled_filled.astype(np.double) - binary_erosion(scaled_filled, kernelX)
        ient_homogenity(scaled_contour,scaled_filled)
        

<div class="alert rwth-feedback">

    
# Feedback:

Liebe TeilnehmerInnen,

Wir würden uns freuen, wenn ihr am Ende jeder Aufgabe kurz eure Meinung aufschreibt. Ihr könnt auf die dadrunter liegende Zelle zu greifen und eure Anmerkungen zu der Aufgabe (oder auch generelles) reinschreiben.


</div>

In [ ]:
rwth_feedback.rwth_feedback('Feedback V6.1', [
    {'id': 'likes', 'type': 'free-text', 'label': 'Das war gut:'}, 
    {'id': 'dislikes', 'type': 'free-text', 'label': 'Das könnte verbessert werden:'}, 
    {'id': 'misc', 'type': 'free-text', 'label': 'Was ich sonst noch sagen möchte:'}, 
    {'id': 'learning', 'type': 'scale', 'label' : 'Ich habe das Gefühl etwas gelernt zu haben.'},
    {'id': 'supervision', 'type': 'scale', 'label' : 'Die Betreuung des Versuchs war gut.'},
    {'id': 'script', 'type': 'scale', 'label' : 'Die Versuchsunterlagen sind verständlich.'},
], "feedback.json", 'pti@ient.rwth-aachen.de')

# >>Weiter zu  [__Teil 2__](Teil_2_old.ipynb)
